In [1]:
import copy
import datetime
import json
import os
import scipy.spatial
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import meshio

import celeri
celeri = reload(celeri)

In [9]:
meshes = {}

mesh_param_filename = './data/test_mesh_param.json'
with open(mesh_param_filename, 'r') as f:
    mesh_param = json.load(f)
for i in range(len(mesh_param)):
     meshes[i] = meshio.read(mesh_param[i]["mesh_filename"])
     meshes[i].verts = meshes[i].get_cells_type("triangle")
    # Expand mesh coordinates
    meshes[i].lon1 = meshes[i].points[meshes[i].v[:,0],0]
    meshes[i].lon2 = meshes[i].points[meshes[i].v[:,1],0]
    meshes[i].lon3 = meshes[i].points[meshes[i].v[:,2],0]
    meshes[i].lat1 = meshes[i].points[meshes[i].v[:,0],1]
    meshes[i].lat2 = meshes[i].points[meshes[i].v[:,1],1]
    meshes[i].lat3 = meshes[i].points[meshes[i].v[:,2],1]
    meshes[i].dep1 = meshes[i].points[meshes[i].v[:,0],2]
    meshes[i].dep2 = meshes[i].points[meshes[i].v[:,1],2]
    meshes[i].dep3 = meshes[i].points[meshes[i].v[:,2],2]
    meshes[i].centroids = np.mean(meshes[i].points[meshes[i].v, :], axis=1)
    # Cartesian coordinates in meters
    meshes[i].x1, meshes[i].y1, meshes[i].z1 = celeri.sph2cart(
        meshes[i].lon1, meshes[i].lat1, celeri.RADIUS_EARTH+1000.*meshes[i].dep1
        )
    meshes[i].x2, meshes[i].y2, meshes[i].z2 = celeri.sph2cart(
        meshes[i].lon2, meshes[i].lat2, celeri.RADIUS_EARTH+1000.*meshes[i].dep2)
    meshes[i].x3, meshes[i].y3, meshes[i].z3 = celeri.sph2cart(
        meshes[i].lon3, meshes[i].lat3, celeri.RADIUS_EARTH+1000.*meshes[i].dep3)
    # Cross products for orientations
    tri_leg1 = np.transpose([np.deg2rad(meshes[i].lon2-meshes[i].lon1), 
         np.deg2rad(meshes[i].lat2-meshes[i].lat1), 
         (1+1000.*meshes[i].dep2/celeri.RADIUS_EARTH)-(1+1000.*meshes[i].dep1/celeri.RADIUS_EARTH)])
    tri_leg2 = np.transpose([np.deg2rad(meshes[i].lon3-meshes[i].lon1), 
         np.deg2rad(meshes[i].lat3-meshes[i].lat1), 
         (1+1000.*meshes[i].dep3/celeri.RADIUS_EARTH)-(1+1000.*meshes[i].dep1/celeri.RADIUS_EARTH)])
    meshes[i].nv = np.cross(tri_leg1, tri_leg2)
    azimuth, elevation, r = celeri.cart2sph(meshes[i].nv[:,0], meshes[i].nv[:,1], meshes[i].nv[:,2])
    meshes[i].strike = celeri.wrap2360(-np.rad2deg(azimuth))
    meshes[i].dip = 90 - np.rad2deg(elevation)
    meshes[i].dip_flag = meshes[i].dip != 90

# print(meshes[i].dip_flag[0:5])
print(meshes[1].verts[0:5])

[[101  36 132]
 [144  81  82]
 [ 83 144  82]
 [109 145 110]
 [149  81 144]]
